## 필수 준비사항

시작하기 전에 다음의 요건이 충족되는지 확인하세요:

- [Python 3.12](https://www.python.org/downloads/) **(권장)** — 이 워크숍은 agent-framework==1.0.0b260123 기반으로 작성되었으며 Python 3.12에서 최적으로 실행됩니다. Python 3.10 이상도 동작할 수 있으나, 호환성 측면에서 Python 3.12 사용을 강력히 권장합니다.
- [Azure CLI 설치](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) 그리고 [인증 (Azure 자격증명 인증)](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli)
  -이 실습에서는 [인터랙티브한 브라우저 로그인 방법](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively?view=azure-cli-latest#sign-in-with-a-browser)을 CLI 인증 권장 방법으로 사용합니다. 터미널에서 `az login` 명령어를 실행하면 브라우저가 열리고 Azure 자격증명으로 로그인할 수 있습니다.


### 1. 리포지토리 클론

```bash
git clone https://github.com/taeyo-kim/MAF_Workshop.git  
cd MAF_Workshop
```

### 2. 가상환경 생성 및 활성화 

> [!IMPORTANT]
>
> **Python 3.12**로 가상 환경을 만드는 것이 `agent-framework==1.0.0b260123`와의 최적 호환성을 위해 권장됩니다. 이 실습은 MAF V2 1.0.0b260123 기반으로 작성되었습니다.

#### 2.1 Visual Studio Code에서 가상환경 생성
- Visual Studio Code 터미널에서 Ctrl + Shift + P를 눌러서 프롬프트가 나오면 Python: Select Interpreter를 선택합니다.
- Create Virtual Environment를 선택하고, Python 3.12 인터프리터를 선택하여 새롭게 .venv 환경을 만듭니다.

------

### 3. Upgrade pip (recommended)

In [ ]:
pip install --upgrade pip

### 4. Agent Framework 설치

이번 워크샵에서 사용할 Agent Framework v2를 설치합니다.

In [ ]:
## 설치에는 약 3-4분 이상 걸릴 수 있습니다.
# 현재 Python 환경 및 agent-framework 설치 확인
import sys
print(f"Python 실행 경로: {sys.executable}")
print(f"Python 버전: {sys.version}")

# agent-framework 설치 시도
%pip install agent-framework==1.0.0b260123
#pip install agent-framework --pre

> [!IMPORTANT] `No module named 'pywintypes'` 에러가 발생하는 경우
>
>이 경우, pywin32의 post-install 스크립트를 수동으로 실행해야 합니다:  
>  
> ** PowerShell 터미널에서 직접 실행:**
> 
> ```powershell
> pip uninstall pywin32 -y; pip install pywin32
>
> .venv\Scripts\python.exe .venv\Scripts\pywin32_postinstall.py -install
> ```
>  
>실행 후 **반드시 Jupyter 커널을 재시작**하세요 (Ctrl+Shift+P → "Restart Kernel")

##### 4.1 Agent Framework API 확인

설치된 agent-framework의 실제 클래스와 메서드를 확인합니다.

In [ ]:
# agent_framework 패키지의 실제 구조 확인
import agent_framework
from agent_framework.azure import AzureOpenAIChatClient

print("=== agent_framework 모듈 확인 ===")
print(f"agent_framework 버전: {agent_framework.__version__ if hasattr(agent_framework, '__version__') else 'N/A'}")
print(f"\nagent_framework 모듈 내용:")
print([x for x in dir(agent_framework) if not x.startswith('_')])

print("\n=== AzureOpenAIChatClient 클래스 확인 ===")
print("사용 가능한 메서드:")
methods = [m for m in dir(AzureOpenAIChatClient) if not m.startswith('_')]
for method in methods:
    print(f"  - {method}")

print("\n=== AzureOpenAIChatClient 초기화 파라미터 확인 ===")
import inspect
sig = inspect.signature(AzureOpenAIChatClient.__init__)
print(f"파라미터: {sig}")

### 5. Microsoft Foundry 프로젝트 생성 및 모델 배포

1. [Microsoft Foundry Portal에서 새로운 프로젝트 만들기](https://learn.microsoft.com/ko-kr/azure/ai-foundry/how-to/create-projects?view=foundry-classic&tabs=foundry)
2. [프로젝트에 Azure OpenAI 모델 배포(GPT-4.1)](https://learn.microsoft.com/en-us/azure/ai-foundry/foundry-models/how-to/deploy-foundry-models?view=foundry-classic)
3. [사용자가 Azure OpenAI 리소스에 대해 Cognitive Services OpenAI User 또는 Cognitive Services OpenAI Contributor 역할을 보유해야 합니다](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/role-based-access-control)
4. 배포된 모델의 엔드포인트 및 키 확인(별도로 복사해 둡니다)

### 6. .env 파일 생성

>**# AI Search 설정 기본값**  
>AZURE_SEARCH_ENDPOINT=https://<자신의 엔드포인트>.search.windows.net  
>AZURE_SEARCH_INDEX_NAME=hotels-sample-index  
>  
>**# MAF 설정 기본값**  
>AZURE_OPENAI_ENDPOINT=https://<자신의 엔드포인트>  
>AZURE_OPENAI_CHAT_DEPLOYMENT_NAME=gpt-4.1  
>OPENAI_RESPONSES_MODEL_ID=gpt-4.1 
>
>
>**# Aspire Dashboard 설정 기본값**  
>#https://learn.microsoft.com/en-us/agent-framework/user-guide/observability?pivots=programming-language-python#setting-up-aspire-dashboard-with-docker    
>ENABLE_INSTRUMENTATION=true  
>OTEL_EXPORTER_OTLP_ENDPOINT=http://localhost:4317  
>  
>**# Foundry 프로젝트 설정 기본값**  
>AZURE_AI_PROJECT_ENDPOINT=https://<자신의 프로젝트 엔드포인트>   
>AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4.1  
